In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_predict, StratifiedKFold
from scipy.stats import randint,uniform
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay,f1_score
from sklearn.model_selection import RandomizedSearchCV
from imblearn import FunctionSampler
from sklearn.utils import resample
from mlxtend.classifier import EnsembleVoteClassifier
from sklearn.base import clone

In [2]:
df= pd.read_csv('./CSC_classified_final_nway_red.csv')
df

,Unnamed: 0,CSC_CSC_ID,CSC_var_inter_prob_b,CSC_var_inter_sigma_b,CSC_var_inter_index_b,CSC_ks_intra_prob_b,CSC_kp_intra_prob_b,CSC_var_intra_index_b,CSC_var_intra_prob_b,CLASS_ID,CLASS_Class,CLASS_Source,dist_bayesfactor,dist_post,p_i,CSC_u/b,CSC_s/b,CSC_m/b,CSC_h/b
0,0,2CXO J171941.7-355418,NaN,NaN,-32768,0.110753,0.434036,0,0.479545,6019018,YSO,SIMBAD,10.624448,9.998668e-01,0.999985,NaN,NaN,0.034251,0.993514
1,1,2CXO J171919.8-355632,0.688725,2.157409e-07,5,0.940708,0.821869,2,0.749379,6018991,YSO,SIMBAD,9.840393,9.991906e-01,1.000000,0.0,0.000000,0.892446,0.000000
2,2,2CXO J171951.2-355325,0.563362,4.632072e-08,3,0.374751,0.604095,0,0.417616,6019047,YSO,SIMBAD,11.270465,9.999699e-01,1.000000,NaN,0.223416,0.437766,0.373852
3,3,2CXO J171956.0-355451,0.609957,5.897142e-08,4,0.379334,0.197464,0,0.369797,6019068,YSO,SIMBAD,11.659839,9.999877e-01,1.000000,NaN,0.094358,0.288578,0.693396
4,4,2CXO J171958.2-355333,0.702705,1.944157e-07,5,0.351655,0.315327,0,0.349900,6019089,YSO,SIMBAD,11.304786,9.999722e-01,1.000000,NaN,0.051214,0.077666,0.827113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47186,47186,2CXO J174626.1-284311,NaN,NaN,-32768,NaN,NaN,-32768,NaN,5944010,YSO,Michael A. Kuhn et al 2021 ApJS 254 33,9.396320,9.978077e-01,1.000000,NaN,NaN,NaN,NaN
47187,47187,2CXO J174549.4-284917,NaN,NaN,-32768,NaN,NaN,-32768,NaN,5433896,STAR,"(Skiff, 2009-2013)",7.274583,7.747192e-01,1.000000,NaN,NaN,NaN,NaN
47188,47188,2CXO J174539.0-290000,NaN,NaN,-32768,NaN,NaN,-32768,NaN,5813290,STAR,SIMBAD,-11.061596,1.585775e-18,1.000000,NaN,NaN,NaN,NaN
47189,47189,2CXO J174539.5-290059,NaN,NaN,-32768,NaN,NaN,1,NaN,5813379,STAR,SIMBAD,-101.120926,0.000000e+00,1.000000,NaN,NaN,NaN,NaN


In [3]:
df.columns

Index(['Unnamed: 0', 'CSC_CSC_ID', 'CSC_var_inter_prob_b',
       'CSC_var_inter_sigma_b', 'CSC_var_inter_index_b', 'CSC_ks_intra_prob_b',
       'CSC_kp_intra_prob_b', 'CSC_var_intra_index_b', 'CSC_var_intra_prob_b',
       'CLASS_ID', 'CLASS_Class', 'CLASS_Source', 'dist_bayesfactor',
       'dist_post', 'p_i', 'CSC_u/b', 'CSC_s/b', 'CSC_m/b', 'CSC_h/b'],
      dtype='object')

In [4]:
ft=df.values[:,[2,3,4,5,6,7,8,15,16,17,18]]
ft

array([[nan, nan, -32768, ..., nan, 0.0342510233907318,
        0.9935137403120856],
       [0.68872468581017, 2.1574094492893e-07, 5, ..., 0.0,
        0.8924463542298487, 0.0],
       [0.5633617454848601, 4.6320716800618e-08, 3, ...,
        0.2234159989734346, 0.437766312091542, 0.373851719840085],
       ...,
       [nan, nan, -32768, ..., nan, nan, nan],
       [nan, nan, -32768, ..., nan, nan, nan],
       [0.970152595457372, 5.5751584932259e-06, 6, ...,
        0.2311717535287629, 0.0844134603451815, 0.5519425431520539]],
      dtype=object)

In [5]:
cl=df.values[:,10]

In [6]:
def normalise(ft):
    l,b= ft.shape
    for i in range(b):
        ft[:,i]= (ft[:,i]-np.nanmin(ft[:,i]))/(np.nanmax(ft[:,i])-np.nanmin(ft[:,i]))
    return ft

def high_neg_impute(ft):
    l,b= ft.shape
    for i in range(b):
        for j in range(l):
            if np.isnan(ft[j,i]):
                ft[j,i]= -1e+8
    return ft

In [7]:
ft=normalise(ft)
ft=high_neg_impute(ft)

/tmp/ipykernel_18638/43106251.py:4: RuntimeWarning: invalid value encountered in divide
  ft[:,i]= (ft[:,i]-np.nanmin(ft[:,i]))/(np.nanmax(ft[:,i])-np.nanmin(ft[:,i]))


In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(cl)
y=le.transform(cl)

In [9]:
ft

array([[-100000000.0, -100000000.0, 0.0, ..., -100000000.0, 0.0, 0.0],
       [0.68872468581017, 1.1676860376727292e-05, 0.9999084696119112,
        ..., 0.0, 0.0, 0.0],
       [0.5633617454848601, 2.507074790530046e-06, 0.9998474493531853,
        ..., 0.0, 0.0, 0.0],
       ...,
       [-100000000.0, -100000000.0, 0.0, ..., -100000000.0, -100000000.0,
        -100000000.0],
       [-100000000.0, -100000000.0, 0.0, ..., -100000000.0, -100000000.0,
        -100000000.0],
       [0.970152595457372, 0.00030175266966890894, 0.9999389797412741,
        ..., 0.0, 0.0, 0.0]], dtype=object)

In [10]:
cl

array(['YSO', 'YSO', 'YSO', ..., 'STAR', 'STAR', 'STAR'], dtype=object)

In [11]:
def remove_missing_data_effects(X,Y):
    X_neg= X
    X_neg[X_neg<0]= -1e+8
    return X_neg,Y

In [12]:
y[cl=="AGN"]

array([0, 0, 0, ..., 0, 0, 0])

In [13]:
np.unique(y)

array([0, 1, 2, 3, 4, 5, 6, 7])

In [14]:
def break_training_data(X,Y):
    r,c=X.shape
    XY= np.empty((r,c+1))
    for i in range(c):
        XY[i,0:-1]=X[i,:]
        XY[i,-1]=Y[i]
    XY_nomaj= XY[Y!=0]
    print(XY_nomaj)
    XY_maj= XY[Y==0]
    print(np.sum(Y==0))
    print(XY_maj)
    n=4
    # print(X.shape,XY.shape)
    XY_maj_chunks= np.array_split(XY_maj,n)
    # print(X)
    # print(Y)    
    # print(XY)
    X_chunks=[]
    Y_chunks=[]
    for i in range(n):
        chunk=XY_maj_chunks[i]
        # print(chunk)
        X_chunks_temp= chunk[:,0:-1]
        # print(X_chunks_temp)
        X_chunks_temp= np.append(X_chunks_temp,XY_nomaj[:,0:-1],axis=0)
        Y_chunks_temp= chunk[:,-1].astype('int')
        Y_chunks_temp= np.append(Y_chunks_temp,XY_nomaj[:,-1].astype('int'),axis=0)
        X_chunks=X_chunks+[X_chunks_temp]
        Y_chunks=Y_chunks+[Y_chunks_temp]
    return X_chunks, Y_chunks

In [21]:
class CustomEnsemble():
    def __init__(self, base_estimator=RandomForestClassifier(), n_estimators=100, 
		max_depth=10,max_features=0.5, min_samples_leaf=10,n_classifiers=4):
    
        self._estimator_type = "classifier"
        self.base_estimator = base_estimator
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.n_classifiers = n_classifiers
        self.estimators = self._generate_estimators()
        
   
    def _generate_estimators(self):
        estimators = []

        for i in range(self.n_classifiers):
            est = clone(self.base_estimator)
    
            est.n_estimators =self.n_estimators
            est.max_depth = self.max_depth
            est.min_samples_leaf = self.min_samples_leaf
            est.max_features= self.max_features
            pipe = Pipeline([
                ('smote', SMOTE(sampling_strategy='auto')),
                ('missing_data_effects_remove', FunctionSampler(func=remove_missing_data_effects, validate = False)),
                ('rf', est)
            ])
            estimators.append(pipe)
        return estimators
        
    def _divide_n_train(self,X,y):
        X_ch,y_ch= break_training_data(X,y)
        for i in range(self.n_classifiers):
            # print(self.estimators[i][1],X_ch[i],y_ch[i])
            self.estimators[i].fit(X_ch[i],y_ch[i])


    
    def fit(self, X, y):
        return self._divide_n_train(X,y)


    def predict_proba(self, X):
        """Compute weighted probabilities for each class."""
        weighted_probas = np.zeros_like(self.estimators[0].predict_proba(X))
        total_weight = self.n_classifiers

        for clf in self.estimators:
            weighted_probas += clf.predict_proba(X)

        return weighted_probas / total_weight

    def predict(self, X):
        """Predict the class labels based on the highest probability."""
        probas = self.predict_proba(X)
        return np.argmax(probas, axis=1)

    def get_params(deep=False):
        return {'max_depth': self.max_depth,
                'n_estimators': self.n_estimators,
                'min_samples_leaf': self.min_samples_leaf,
                'max_features': self.max_features}
    def set_params(n_estimators=100,max_depth=10,max_features=0.5, min_samples_leaf=10):
        est.n_estimators =self.n_estimators
        est.max_depth = self.max_depth
        est.min_samples_leaf = self.min_samples_leaf
        est.max_features= self.max_features

In [16]:
# import numpy as np
# from sklearn.base import BaseEstimator, ClassifierMixin
# from sklearn.datasets import make_classification
# from sklearn.model_selection import train_test_split


# class SoftVotingClassifier(BaseEstimator, ClassifierMixin):
#     def __init__(self, classifiers, weights=None):
#         self.classifiers = classifiers
#         self.weights = weights if weights else [1] * len(classifiers)

#     def fit(self, X, y):
#         """Fit all classifiers to the data."""
#         for clf in self.classifiers:
#             clf.fit(X, y)
#         return self

#     def predict_proba(self, X):
#         """Compute weighted probabilities for each class."""
#         weighted_probas = np.zeros_like(self.classifiers[0].predict_proba(X))
#         total_weight = sum(self.weights)

#         for clf, weight in zip(self.classifiers, self.weights):
#             weighted_probas += weight * clf.predict_proba(X)

#         return weighted_probas / total_weight

#     def predict(self, X):
#         """Predict the class labels based on the highest probability."""
#         probas = self.predict_proba(X)
#         return np.argmax(probas, axis=1)

# # Create the SoftVotingClassifier
# voting_clf = SoftVotingClassifier(classifiers=[clf1, clf2, clf3, clf4], weights=[1, 1, 1, 1])

In [22]:
ens= CustomEnsemble()
ens.fit(ft,y)

[[-1.00000000e+08 -1.00000000e+08  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  7.00000000e+00]
 [ 6.88724686e-01  1.16768604e-05  9.99908470e-01 ...  0.00000000e+00
   0.00000000e+00  7.00000000e+00]
 [ 5.63361745e-01  2.50707479e-06  9.99847449e-01 ...  0.00000000e+00
   0.00000000e+00  7.00000000e+00]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]
28869
[[5.37958507e-316 6.36598743e-313 7.21478574e-313 ... 5.37956462e-316
  5.37956462e-316 5.37956462e-316]
 [6.26067506e-310 5.37958591e-316 1.27319747e-313 ... 2.11954162e-321
  6.26067438e-310 4.24399158e-314]
 [5.37961116e-316 6.26068009e-310 6.26068009e-310 ... 5.37956462e-316
  5.37956462e-316 5.37956462e-316]
 ...
 [0.00000000e+000 0.00000000e+000 0.00

/home/asm/miniconda3/lib/python3.12/site-packages/sklearn/utils/_array_api.py:751: RuntimeWarning: overflow encountered in cast
  array = numpy.asarray(array, order=order, dtype=dtype)


ValueError: Input X contains infinity or a value too large for dtype('float32').

In [23]:
ens.predict(ft)

AttributeError: 'RandomForestClassifier' object has no attribute 'estimators_'

In [19]:
y

array([7, 7, 7, ..., 5, 5, 5])

In [20]:
# Use StratifiedKFold for cross-validation
cv = StratifiedKFold(n_splits=20, shuffle=True)

# Train the LightGBM model using cross-validation with SMOTE
y_pred = cross_val_predict(ens, ft, y,cv=cv)

TypeError: CustomEnsemble.get_params() got an unexpected keyword argument 'deep'